In [1]:
from typing import Tuple, List
import polars as pl
from pprint import pprint
import sys
from pathlib import Path
import psutil
import pandas as pd
import time
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from dateutil.relativedelta import relativedelta
from matplotlib.ticker import MultipleLocator

plt.rcParams["font.family"] = "Freesentation"
plt.rcParams["axes.unicode_minus"] = False


# 이상치 탐지

## 목표
- MAUDE 데이터에서 defect_type(키워드) 이상치 탐지하기
- 최근 달에 갑자기 증가한 키워드 찾기



In [2]:
# 데이터 로드
lf = pl.scan_parquet("../data/maude.parquet")

# 기본 정보 확인
total_rows = lf.select(pl.len()).collect().item()
print(f"전체 행 수: {total_rows:,}개")


전체 행 수: 509,192개


In [7]:
lf.collect_schema()

Schema([('mdr_report_key', Int32),
        ('adverse_event_flag', Boolean),
        ('product_problem_flag', Boolean),
        ('date_occurred', Date),
        ('date_received', Date),
        ('date_manufactured', Date),
        ('event_type', Categorical),
        ('previous_use_flag', Boolean),
        ('single_use_flag', Boolean),
        ('reprocessed_and_reused_flag', Boolean),
        ('product_problems', String),
        ('manufacturer_name', String),
        ('brand_name', String),
        ('model_number', String),
        ('udi_di', String),
        ('product_code', String),
        ('operator', Boolean),
        ('product_name', String),
        ('patient_age', Int64),
        ('mdr_text', String),
        ('outcome_L', Boolean),
        ('outcome_H', Boolean),
        ('outcome_S', Boolean),
        ('outcome_C', Boolean),
        ('outcome_R', Boolean),
        ('outcome_D', Boolean),
        ('outcome_O', Boolean),
        ('patient_harm', String),
        ('problem_compo

# 이건 일단 넣어둬..

## 1. 날짜 처리

date_received 컬럼에서 월 정보를 추출


In [ ]:
def extract_month_from_date(lf, date_column):
    """
    날짜 컬럼에서 월 정보를 추출하는 함수
    
    Parameters:
    lf: polars LazyFrame
    date_column: 날짜 컬럼 이름 (예: 'date_received')
    
    Returns:
    월 정보가 추가된 LazyFrame (as_of_month 컬럼 추가)
    """
    result = lf.with_columns([
        pl.col(date_column).cast(pl.Date).dt.strftime("%Y-%m").alias("as_of_month")
    ])
    return result


In [ ]:
# 월 정보 추출
lf_with_month = extract_month_from_date(lf, "date_received")

# 확인
sample = lf_with_month.select(["date_received", "as_of_month"]).head(10).collect()
print("월 정보 추출 확인:")
print(sample.to_pandas())


## 2) 키워드 테이블 만들기

defect_type을 키워드로 사용


In [ ]:
def create_keyword_table(lf, keyword_column, month_column):
    """
    키워드 테이블 생성 함수 (report-level presence)
    defect_type의 각 값을 개별 keyword로 처리
    
    Parameters:
    lf: polars LazyFrame
    keyword_column: 키워드 컬럼 이름 (예: 'defect_type')
    month_column: 월 컬럼 이름 (예: 'as_of_month')
    
    Returns:
    키워드 테이블 LazyFrame (mdr_report_key, keyword, month)
    """
    result = (
        lf
        .filter(pl.col(keyword_column).is_not_null())
        .select([
            pl.col("mdr_report_key"),
            pl.col(keyword_column).alias("keyword"),
            pl.col(month_column).alias("month")
        ])
        .unique(subset=["mdr_report_key", "keyword", "month"])
    )
    return result

In [ ]:
# 키워드 테이블 생성
lf_keyword_table = create_keyword_table(lf_with_month, "defect_type", "as_of_month")

print("\n키워드 테이블 샘플:")
keyword_sample = lf_keyword_table.collect()
print(keyword_sample.to_pandas())

## 4. 월별 집계

두 가지를 집계해야 함:
1. 월별 키워드별 보고서 수 (C) 
2. 월별 전체 보고서 수 (N) 

In [ ]:
def monthly_keyword_counts(lf_keyword_table, month_column):
    """
    월별 키워드별 보고서 수 집계 함수
    
    Parameters:
    lf_keyword: 키워드 테이블 LazyFrame (keyword, month 포함)
    month_column: 월 컬럼 이름 (예: 'month')
    
    Returns:
    월별 키워드별 집계 결과 LazyFrame
    - keyword: 키워드
    - month: 월
    - n_reports_keyword: 그 달에 그 키워드를 포함한 보고서 수
    """
    result = (
        lf_keyword_table
        .group_by(["keyword", month_column])
        .agg([
            pl.col("mdr_report_key").n_unique().alias("n_reports_keyword")
        ])
        .sort(["keyword", month_column])
    )
    return result


In [ ]:
# 월별 키워드별 집계
lf_keyword_monthly = monthly_keyword_counts(lf_keyword_table, "month")

print("월별 키워드별 집계 결과 (샘플):")
keyword_monthly_sample = lf_keyword_monthly.head(20).collect()
print(keyword_monthly_sample.to_pandas())


In [ ]:
def monthly_total_reports(lf, date_column):
    """
    월별 전체 보고서 수 집계 함수
    
    Parameters:
    lf: 원본 LazyFrame
    date_column: 날짜 컬럼 이름 (예: 'date_received')
    
    Returns:
    월별 전체 보고서 수 LazyFrame
    - month: 월
    - n_total_reports: 그 달 전체 보고서 수
    """
    result = (
        lf
        .filter(pl.col(date_column).is_not_null())
        .with_columns([
            pl.col(date_column).cast(pl.Date).dt.strftime("%Y-%m").alias("month")
        ])
        .group_by("month")
        .agg([
            pl.len().alias("n_total_reports")
        ])
        .sort("month")
    )
    return result


In [ ]:
# 월별 전체 보고서 수 집계
lf_monthly_total = monthly_total_reports(lf, "date_received")

print("월별 전체 보고서 수:")
monthly_total = lf_monthly_total.collect()
print(monthly_total.to_pandas())


In [ ]:
def calculate_keyword_ratio_time_series(lf_keyword_monthly, lf_monthly_total, month_column):
    """
    키워드별 비율 시계열 계산 함수
    
    Parameters:
    lf_keyword_monthly: 월별 키워드별 집계 LazyFrame
    lf_monthly_total: 월별 전체 보고서 수 LazyFrame
    month_column: 월 컬럼 이름 (예: 'month')
    
    Returns:
    비율 시계열 LazyFrame
    - keyword: 키워드
    - month: 월
    - n_reports_keyword: 그 달에 그 키워드를 포함한 보고서 수
    - n_total_reports: 그 달 전체 보고서 수
    - p_t: 비율 (n_reports_keyword / n_total_reports)
    """
    result = (
        lf_keyword_monthly
        .join(lf_monthly_total, on=month_column, how="left")
        .with_columns([
            (pl.col("n_reports_keyword") / pl.col("n_total_reports")).alias("p_t")
        ])
        .sort(["keyword", month_column])
    )
    return result


In [ ]:
# 비율 시계열 계산
lf_keyword_ratio = calculate_keyword_ratio_time_series(lf_keyword_monthly, lf_monthly_total, "month")

print("키워드별 비율 시계열 (샘플):")
keyword_ratio_sample = lf_keyword_ratio.head(20).collect()
print(keyword_ratio_sample.to_pandas())


## 3) 베이스라인 비교를 위한 윈도우 설정

### 개념 정리 (헷갈려서 다시 정리)
- **최근 윈도우**: 지금 상황 (최근 1개월 or 3개월)
- **기준 윈도우**: 과거 비교 대상 (직전 1개월 or 3개월)

윈도우 구성 :

    - 최근 윈도우 구성 
        - 1개월 : 최근 완료된 1개월 (ex. 2025-11)
        - 3개월 : 최근 완료된 3개월 (ex. 2025-11, 10, 9)

    - 기준 윈도우
        - 직전 1월 / 직전 3개월 (ex. 2025-10 / 10, 9, 8)
    
    - 슬라이딩 offset은 1개월로 지정

    - `C_recent` = 최근기간 키워드 보고서 합
    - `C_base` = 기준기간 키워드 보고서 합      
    - `N_recent` = 최근기간 전체 보고서 합
    - `N_base` = 기준기간 전체 보고서 합

In [ ]:
def create_window_aggregates_by_keyword(lf_keyword_monthly, lf_monthly_total, month_column, window_size):
    """
    키워드별 윈도우 집계 함수 (최근 윈도우와 기준 윈도우)
    create_complete_baseline_table과 동일한 윈도우 계산 방식 및 데이터 소스 사용
    
    Parameters:
    lf_keyword_monthly: 키워드별 월별 집계 LazyFrame (keyword, month, n_reports_keyword)
    lf_monthly_total: 월별 전체 보고서 수 LazyFrame
    month_column: 월 컬럼 이름
    window_size: 윈도우 크기 (1 또는 3)
    
    Returns:
    키워드별 윈도우 집계 결과 LazyFrame
    - keyword: 키워드
    - C_recent: 최근 k개월의 n_reports_keyword 합
    - N_recent: 최근 k개월의 n_total_reports 합 (키워드 무관, 전체 보고서 수)
    - C_base: 직전 k개월의 n_reports_keyword 합
    - N_base: 직전 k개월의 n_total_reports 합 (키워드 무관, 전체 보고서 수)
    """
    from datetime import datetime
    from dateutil.relativedelta import relativedelta
    
    # 각 키워드별로 처리
    # create_complete_baseline_table과 동일한 데이터 소스 사용
    df_keyword_monthly = lf_keyword_monthly.collect()
    df_monthly_total = lf_monthly_total.collect()
    
    # 전체 보고서 수에서 최근/기준 윈도우 계산 (키워드와 무관)
    df_monthly_sorted = df_monthly_total.sort(month_column, descending=True)
    
    if len(df_monthly_sorted) < window_size * 2:
        return pl.DataFrame({
            'keyword': [],
            'C_recent': [],
            'N_recent': [],
            'C_base': [],
            'N_base': []
        }).lazy()
    
    # create_complete_baseline_table과 동일한 방식으로 최신 월 가져오기
    # Polars DataFrame에서 최신 월 가져오기
    as_of_month = df_monthly_sorted.select(month_column).to_series()[0]
    as_of_date = datetime.strptime(as_of_month, "%Y-%m")
    
    # 최근 기간 계산 (create_complete_baseline_table과 동일)
    if window_size == 1:
        recent_months = [as_of_month]
        base_months = [(as_of_date - relativedelta(months=1)).strftime("%Y-%m")]
    else:  # window_size == 3
        recent_months = [
            (as_of_date - relativedelta(months=i)).strftime("%Y-%m")
            for i in range(3)
        ]
        base_months = [
            (as_of_date - relativedelta(months=i)).strftime("%Y-%m")
            for i in range(1, 4)
        ]
    
    # 전체 보고서 수 집계 (키워드와 무관)
    # create_complete_baseline_table과 동일한 방식으로 집계
    recent_months_data = df_monthly_sorted.filter(pl.col(month_column).is_in(recent_months))
    N_recent = recent_months_data.select(pl.col('n_total_reports').sum()).item()
    
    base_months_data = df_monthly_sorted.filter(pl.col(month_column).is_in(base_months))
    N_base = base_months_data.select(pl.col('n_total_reports').sum()).item()
    
    results = []
    
    for keyword in df_keyword_monthly['keyword'].unique():
        keyword_data = (
            df_keyword_monthly
            .filter(pl.col("keyword") == keyword)
            .sort(month_column, descending=True)
        )
        
        # 최근 윈도우: recent_months에 해당하는 데이터
        recent_data = keyword_data.filter(pl.col(month_column).is_in(recent_months))
        C_recent = recent_data['n_reports_keyword'].sum() if len(recent_data) > 0 else 0
        
        # 기준 윈도우: base_months에 해당하는 데이터
        base_data = keyword_data.filter(pl.col(month_column).is_in(base_months))
        C_base = base_data['n_reports_keyword'].sum() if len(base_data) > 0 else 0
        
        results.append({
            'keyword': keyword,
            'C_recent': C_recent,
            'N_recent': N_recent,  # 키워드와 무관한 전체 보고서 수
            'C_base': C_base,
            'N_base': N_base  # 키워드와 무관한 전체 보고서 수
        })
    
    # 결과를 LazyFrame으로 변환
    if results:
        result = pl.DataFrame(results).lazy()
    else:
        result = pl.DataFrame({
            'keyword': [],
            'C_recent': [],
            'N_recent': [],
            'C_base': [],
            'N_base': []
        }).lazy()
    
    return result


In [ ]:
# 1개월 윈도우 집계 (키워드별)
# create_complete_baseline_table과 동일한 데이터 소스 사용
lf_window_1m = create_window_aggregates_by_keyword(lf_keyword_monthly, lf_monthly_total, "month", 1)

print("1개월 윈도우 집계 (키워드별):")
window_1m = lf_window_1m.collect()
print(window_1m.to_pandas())


In [ ]:
# # 윈도우 계산 확인 (디버깅)
# print("=" * 60)
# print("윈도우 계산 확인")
# print("=" * 60)

# # 데이터의 최신 월 확인
# latest_months = lf_monthly_total.sort("month", descending=True).head(6).collect()
# print("\n데이터의 최신 6개월:")
# print(latest_months.to_pandas())

# print("\n" + "=" * 60)


In [ ]:
# 3개월 윈도우 집계 (키워드별)
# create_complete_baseline_table과 동일한 데이터 소스 사용
lf_window_3m = create_window_aggregates_by_keyword(lf_keyword_monthly, lf_monthly_total, "month", 3)

print("3개월 윈도우 집계 (키워드별):")
window_3m = lf_window_3m.collect()
print(window_3m.to_pandas())


## 4) 절대 빈도 고려 및 비율 계산

- 하드 가드: C_recent >= 20
- 비율 계산: ratio, p_recent, p_base, score_ratio


In [ ]:
def ratio_score(lf):
    """
    비율과 점수를 계산하는 함수
    
    Parameters:
    lf: 윈도우 집계 결과 LazyFrame (C_recent, N_recent, C_base, N_base 포함)
    
    Returns:
    비율과 점수가 추가된 LazyFrame
    - ratio: 스무딩 포함 비율 (C_recent + 1) / (C_base + 1)
    - p_recent: C_recent / N_recent
    - p_base: C_base / N_base
    - score_ratio: log(ratio) * log(C_recent + 1)
    """
    # 먼저 ratio를 계산
    result = lf.with_columns([
        # 스무딩 포함 비율
        ((pl.col("C_recent") + 1) / (pl.col("C_base") + 1)).round(2).alias("ratio"),
        # p_recent와 p_base
        (pl.col("C_recent") / pl.col("N_recent")).round(2).alias("p_recent"),
        (pl.col("C_base") / pl.col("N_base")).round(2).alias("p_base")
    ])
    
    # ratio가 계산된 후에 score_ratio 계산 (polars에서 이미 계산된 컬럼 참조)
    result = result.with_columns([
        # 가중 점수 계산: log(ratio) * log(C_recent + 1)
        # polars에서 log 계산 (ratio + 1e-10으로 0 방지)
        ((pl.col("ratio")).log() * (pl.col("C_recent") + 1).log()).round(2).alias("score_ratio")
    ])
    
    return result


In [ ]:
# 1개월 윈도우 비율 및 점수 계산
lf_window_1m_with_score = ratio_score(lf_window_1m)
print("1개월 윈도우 비율 및 점수:")
window_1m_with_score = lf_window_1m_with_score.collect()
print(window_1m_with_score.to_pandas())


In [ ]:
# 3개월 윈도우 비율 및 점수 계산
lf_window_3m_with_score = ratio_score(lf_window_3m)

print("3개월 윈도우 비율 및 점수:")
window_3m_with_score = lf_window_3m_with_score.collect()
print(window_3m_with_score.to_pandas())


In [ ]:
def frequency_guard(lf, min_count=20):
    """
    절대 빈도 가드를 적용하는 함수 (C_recent >= min_count)
    
    Parameters:
    lf: 윈도우 집계 결과 LazyFrame
    min_count: 최소 건수 기준 (기본값 20)
    
    Returns:
    가드를 통과한 결과만 필터링된 LazyFrame
    """
    result = lf.filter(pl.col("C_recent") >= min_count)
    return result


In [ ]:
# 절대 빈도 가드 적용 (1개월)
lf_filtered_1m = frequency_guard(lf_window_1m_with_score, min_count=20)

print("1개월 윈도우 - 가드 통과 결과:")
filtered_1m = lf_filtered_1m.collect()
print(filtered_1m.to_pandas())


In [ ]:
# 절대 빈도 가드 적용 (3개월)
lf_filtered_3m = frequency_guard(lf_window_3m_with_score, min_count=20)

print("3개월 윈도우 - 가드 통과 결과:")
filtered_3m = lf_filtered_3m.collect()
print(filtered_3m.to_pandas())


## 5) 상대적 증가율 탐지

ratio >= 2.0 조건을 만족하는 이상치 탐지


In [ ]:
def detect_spike_by_ratio(lf, ratio_threshold=2.0):
    """
    상대적 증가율로 이상치를 탐지하는 함수
    
    Parameters:
    lf: 가드를 통과한 윈도우 집계 결과 LazyFrame
    ratio_threshold: 비율 임계값 (기본값 2.0)
    기본값 2.0 / 보수적인 값 3.0 (선택)
    
    Returns:
    이상치로 탐지된 결과만 필터링된 LazyFrame
    """
    result = lf.filter(pl.col("ratio") >= ratio_threshold)
    return result


In [ ]:
def run_spike_detection(lf_filtered, window_name, ratio_threshold=1.5):
    """
    이상치 탐지를 실행하고 결과를 출력하는 함수
    
    Parameters:
    lf_filtered: 필터링된 LazyFrame
    window_name: 윈도우 이름 (예: "1개월", "3개월")
    ratio_threshold: ratio 기준값
    
    Returns:
    탐지 결과 DataFrame
    """
    lf_spike = detect_spike_by_ratio(lf_filtered, ratio_threshold=ratio_threshold)
    spike_result = lf_spike.collect()
    
    print(f"\n{window_name} 윈도우 이상치 탐지 결과:")
    print(f"이상치로 탐지된 키워드 수: {len(spike_result)}")
    
    if len(spike_result) > 0:
        print(spike_result.to_pandas())
    else:
        print("이상치로 탐지된 키워드가 없습니다.")
    
    return spike_result

# 실행
spike_1m = run_spike_detection(lf_filtered_1m, "1개월")
spike_3m = run_spike_detection(lf_filtered_3m, "3개월")

# 총합본

## 6) 통합 베이스라인 집계 함수

모든 키워드를 포함하여 1개월/3개월 윈도우를 통합 계산하는 함수

In [ ]:
def create_complete_baseline_table(
    lf,
    as_of_month=None,
    spike_threshold_ratio=None,
    min_c_recent=20
):
    """
    베이스라인 집계 테이블을 Long Format으로 생성하는 함수
    모든 keyword(defect_type)에 대해 0이어도 결과에 포함됩니다.
    
    Parameters:
    -----------
    lf_keyword_monthly : pl.LazyFrame
        키워드별 월별 집계 데이터 (keyword, month, n_reports_keyword)
    lf_monthly_total : pl.LazyFrame
        월별 전체 보고서 수 데이터 (month, n_total_reports)
    as_of_month : str, optional
        기준 월 (예: "2025-11"). None이면 자동으로 최신 월 사용
    spike_threshold_ratio : float, default=2.0
        스파이크 판단 기준 비율 (ratio >= spike_threshold_ratio)
    min_c_recent : int, default=20
        스파이크 판단 최소 C_recent 값 (C_recent >= min_c_recent)
    
    Returns:
    --------
    pl.LazyFrame
        Long Format 베이스라인 집계 테이블 (모든 keyword 포함)
    """
    from datetime import datetime
    from dateutil.relativedelta import relativedelta
    import time
    
    # 전체 실행 시간 측정 시작
    start_time = time.time()

    # 2. 월 정보 추출
    lf_with_month = lf.with_columns([
        pl.col("date_received").cast(pl.Date).dt.strftime("%Y-%m").alias("as_of_month")
    ])
    
    # 3. 키워드 테이블 생성
    lf_keyword_table = (
        lf_with_month
        .filter(pl.col("defect_type").is_not_null())
        .select([
            pl.col("mdr_report_key"),
            pl.col("defect_type").alias("keyword"),
            pl.col("as_of_month").alias("month")
        ])
        .unique(subset=["mdr_report_key", "keyword", "month"])
    )
    
    # 4. 월별 키워드별 집계
    lf_keyword_monthly = (
        lf_keyword_table
        .group_by(["keyword", "month"])
        .agg([
            pl.col("mdr_report_key").n_unique().alias("n_reports_keyword")
        ])
        .sort(["keyword", "month"])
    )
    
    # 5. 월별 전체 보고서 수 집계
    lf_monthly_total = (
        lf
        .filter(pl.col("date_received").is_not_null())
        .with_columns([
            pl.col("date_received").cast(pl.Date).dt.strftime("%Y-%m").alias("month")
        ])
        .group_by("month")
        .agg([
            pl.len().alias("n_total_reports")
        ])
        .sort("month")
    )
    
    # as_of_month가 지정되지 않으면 자동으로 최신 월 가져오기
    if as_of_month is None:
        available_months = (
            lf_monthly_total
            .select("month")
            .sort("month", descending=True)
            .collect()
            .to_series()
            .to_list()
        )
        as_of_month = available_months[0] if available_months else None
        if as_of_month is None:
            raise ValueError("사용 가능한 월 데이터가 없습니다.")
        print(f"자동으로 최신 월을 사용합니다: {as_of_month}")
    
    # as_of_month를 기준으로 기간 계산
    as_of_date = datetime.strptime(as_of_month, "%Y-%m")
    
    # 최근 기간 계산
    # ============================================================
    # 베이스라인 윈도우 구성 (상대 증가율 + 검정 공통)
    # ============================================================
    # 최근 윈도우 2종:
    # - 1개월: 최근 완료된 1개월 (예: 11월)
    recent_1month = [as_of_month]
    
    # - 3개월: 최근 완료된 3개월 (예: 11, 10, 9월)
    recent_3month = [
        (as_of_date - relativedelta(months=i)).strftime("%Y-%m")
        for i in range(3)  # i=0: 11월, i=1: 10월, i=2: 9월
    ]
    
    # 기준 윈도우 (직전 기간, 겹치지 않도록):
    # - 직전 1개월: 최근 1개월의 직전 1개월 (예: 10월)
    base_1month = [(as_of_date - relativedelta(months=1)).strftime("%Y-%m")]
    
    # - 직전 3개월: 최근 3개월의 직전 3개월 (예: 10, 9, 8월)
    # recent_3month의 첫 번째 월(10월)부터 직전 3개월
    base_3month = [
        (as_of_date - relativedelta(months=i)).strftime("%Y-%m")
        for i in range(1, 4)  # i=1: 10월, i=2: 9월, i=3: 8월
    ]
    
    # 슬라이딩 offset: 1개월 (최근 윈도우와 기준 윈도우 사이)
    # 키워드별 집계: C_recent, C_base, N_recent, N_base
    
    print(f"\n기준 월: {as_of_month}")
    print(f"최근 1개월: {recent_1month}")
    print(f"최근 3개월: {recent_3month}")
    print(f"기준 1개월: {base_1month}")
    print(f"기준 3개월: {base_3month}")
    
    # 모든 keyword 목록 가져오기 (0이어도 포함하기 위해)
    step0_start = time.time()
    all_keywords = (
        lf_keyword_monthly
        .select("keyword")
        .unique()
        .collect()
    )
    step0_time = time.time() - step0_start
    print(f"[시간 측정] 모든 keyword 목록 수집: {step0_time:.3f}초")
    print(f"총 keyword 개수: {len(all_keywords)}")
    
    # 1개월 윈도우: 키워드별 보고서 수 집계
    step1_start = time.time()
    baseline_1month_keyword_raw = (
        lf_keyword_monthly
        .filter(pl.col("month").is_in(recent_1month + base_1month))
        .with_columns([
            pl.when(pl.col("month").is_in(recent_1month))
            .then(pl.lit("recent"))
            .otherwise(pl.lit("base"))
            .alias("window_type")
        ])
        .group_by(["keyword", "window_type"])
        .agg([
            pl.col("n_reports_keyword").sum().alias("C")
        ])
        .collect()
        .pivot(
            index="keyword",
            on="window_type",
            values="C",
            aggregate_function="first"
        )
        .with_columns([
            pl.col("recent").fill_null(0).alias("C_recent_1m"),
            pl.col("base").fill_null(0).alias("C_base_1m")
        ])
        .select([
            pl.col("keyword"),
            pl.col("C_recent_1m"),
            pl.col("C_base_1m")
        ])
    )
    
    # 모든 keyword를 포함하도록 outer join
    baseline_1month_keyword = (
        all_keywords
        .join(
            baseline_1month_keyword_raw,
            on="keyword",
            how="left"
        )
        .with_columns([
            pl.col("C_recent_1m").fill_null(0),
            pl.col("C_base_1m").fill_null(0)
        ])
    )
    step1_time = time.time() - step1_start
    
    # 1개월 윈도우: 전체 보고서 수 집계
    step2_start = time.time()
    n_total_1month_reports = (
        lf_monthly_total
        .filter(pl.col("month").is_in(recent_1month + base_1month))
        .with_columns(
            pl.when(pl.col("month").is_in(recent_1month))
            .then(pl.lit('recent'))
            .otherwise(pl.lit('base'))
            .alias('period')
        )
        .group_by('period')
        .agg(pl.sum('n_total_reports').alias('N'))
        .collect()
        .to_pandas()
    )
    
    N_recent_1m = n_total_1month_reports[n_total_1month_reports['period'] == 'recent']['N'].values[0] if len(n_total_1month_reports[n_total_1month_reports['period'] == 'recent']) > 0 else 0
    N_base_1m = n_total_1month_reports[n_total_1month_reports['period'] == 'base']['N'].values[0] if len(n_total_1month_reports[n_total_1month_reports['period'] == 'base']) > 0 else 0
    step2_time = time.time() - step2_start
    
    # 3개월 윈도우: 키워드별 보고서 수 집계
    step3_start = time.time()

    # Recent 3개월 집계 (11, 10, 9월)
    recent_3m_keyword = (
        lf_keyword_monthly
        .filter(pl.col("month").is_in(recent_3month))
        .group_by("keyword")
        .agg([
            pl.col("n_reports_keyword").sum().alias("C_recent_3m")
        ])
        .collect()
    )

    # Base 3개월 집계 (10, 9, 8월)
    base_3m_keyword = (
        lf_keyword_monthly
        .filter(pl.col("month").is_in(base_3month))
        .group_by("keyword")
        .agg([
            pl.col("n_reports_keyword").sum().alias("C_base_3m")
        ])
        .collect()
    )

    # 두 결과 조인
    baseline_3month_keyword_raw = (
        recent_3m_keyword
        .join(base_3m_keyword, on="keyword", how="outer")
        .with_columns([
            pl.col("C_recent_3m").fill_null(0),
            pl.col("C_base_3m").fill_null(0)
        ])
    )

    # 모든 keyword를 포함하도록 outer join
    baseline_3month_keyword = (
        all_keywords
        .join(baseline_3month_keyword_raw, on="keyword", how="left")
        .with_columns([
            pl.col("C_recent_3m").fill_null(0),
            pl.col("C_base_3m").fill_null(0)
        ])
    )

    step3_time = time.time() - step3_start
    
    # # 디버깅: 특정 키워드의 base_3month 월별 합계 확인
    # test_keyword = "Electrical/Power"
    # test_keyword_monthly = (
    #     lf_keyword_monthly
    #     .filter((pl.col("keyword") == test_keyword) & (pl.col("month").is_in(base_3month)))
    #     .collect()
    # )
    # if len(test_keyword_monthly) > 0:
    #     manual_sum = test_keyword_monthly["n_reports_keyword"].sum()
    #     print(f"\n[디버깅] {test_keyword}의 base_3month ({base_3month}) 월별 데이터:")
    #     print(test_keyword_monthly.to_pandas())
    #     print(f"[디버깅] {test_keyword}의 base_3month 수동 합계: {manual_sum}")
    #     test_result = baseline_3month_keyword_raw.filter(pl.col("keyword") == test_keyword)
    #     if len(test_result) > 0:
    #         print(f"[디버깅] {test_keyword}의 C_base_3m (함수 결과): {test_result['C_base_3m'][0]}")
    
    # # 모든 keyword를 포함하도록 outer join
    # baseline_3month_keyword = (
    #     all_keywords
    #     .join(
    #         baseline_3month_keyword_raw,
    #         on="keyword",
    #         how="left"
    #     )
    #     .with_columns([
    #         pl.col("C_recent_3m").fill_null(0),
    #         pl.col("C_base_3m").fill_null(0)
    #     ])
    # )
    # step3_time = time.time() - step3_start
    
    # 3개월 윈도우: 전체 보고서 수 집계
    step4_start = time.time()

    # Recent 3개월 집계 (11, 10, 9월)
    N_recent_3m = (
        lf_monthly_total
        .filter(pl.col("month").is_in(recent_3month))
        .select(pl.sum('n_total_reports'))
        .collect()
        .item()
    )

    # Base 3개월 집계 (10, 9, 8월)
    N_base_3m = (
        lf_monthly_total
        .filter(pl.col("month").is_in(base_3month))
        .select(pl.sum('n_total_reports'))
        .collect()
        .item()
    )

    step4_time = time.time() - step4_start
    print(f"[시간 측정] 3개월 전체 보고서 집계: {step4_time:.3f}초")
    
    # 1개월 데이터 준비
    step5_start = time.time()
    baseline_1m = (
        baseline_1month_keyword
        .with_columns([
            pl.lit(as_of_month).alias("as_of_month"),
            pl.lit("1").alias("window"),
            pl.col("C_recent_1m").fill_null(0).cast(pl.Int64).alias("C_recent"),
            pl.col("C_base_1m").fill_null(0).cast(pl.Int64).alias("C_base"),
            pl.lit(N_recent_1m).cast(pl.Int64).alias("N_recent"),
            pl.lit(N_base_1m).cast(pl.Int64).alias("N_base")
        ])
        .select([
            "as_of_month",
            "window",
            "keyword",
            "C_recent",
            "C_base",
            "N_recent",
            "N_base"
        ])
    )
    
    # 3개월 데이터 준비
    baseline_3m = (
        baseline_3month_keyword
        .with_columns([
            pl.lit(as_of_month).alias("as_of_month"),
            pl.lit("3").alias("window"),
            pl.col("C_recent_3m").fill_null(0).cast(pl.Int64).alias("C_recent"),
            pl.col("C_base_3m").fill_null(0).cast(pl.Int64).alias("C_base"),
            pl.lit(N_recent_3m).cast(pl.Int64).alias("N_recent"),
            pl.lit(N_base_3m).cast(pl.Int64).alias("N_base")
        ])
        .select([
            "as_of_month",
            "window",
            "keyword",
            "C_recent",
            "C_base",
            "N_recent",
            "N_base"
        ])
    )
    step5_time = time.time() - step5_start
    
    # Long format으로 통합
    step6_start = time.time()
    
    # Long format으로 통합 및 계산
    threshold = spike_threshold_ratio
    
    # 1단계: ratio와 is_spike 계산
    # 스키마 통일을 위해 모든 숫자 컬럼을 Int64로 캐스팅
    baseline_1m = baseline_1m.with_columns([
        pl.col("C_recent").cast(pl.Int64),
        pl.col("C_base").cast(pl.Int64),
        pl.col("N_recent").cast(pl.Int64),
        pl.col("N_base").cast(pl.Int64)
    ])
    baseline_3m = baseline_3m.with_columns([
        pl.col("C_recent").cast(pl.Int64),
        pl.col("C_base").cast(pl.Int64),
        pl.col("N_recent").cast(pl.Int64),
        pl.col("N_base").cast(pl.Int64)
    ])
    
    baseline_final = (
        pl.concat([baseline_1m, baseline_3m])
        .with_columns([
            # 스무딩 포함 비율 계산: ratio = (C_recent + 1) / (C_base + 1)
            ((pl.col("C_recent") + 1) / (pl.col("C_base") + 1))
            .round(2)
            .alias("ratio"),
            # is_spike 계산 (필터 후보: ratio >= 2 또는 ratio >= 3)
            pl.when(
                (pl.col("C_recent") >= min_c_recent) & 
                (((pl.col("C_recent") + 1) / (pl.col("C_base") + 1)) >= threshold)
            )
            .then(True)
            .otherwise(False)
            .alias("is_spike")
        ])
    )
    
    # 2단계: ratio가 계산된 후에 가중 점수 계산
    baseline_final = (
        baseline_final
        .with_columns([
            # 소프트 가중(랭킹 반영) - 건수가 많을수록 우선순위
            # 방법 1: log(C_recent + 1) 사용
            (pl.col("ratio") * (pl.col("C_recent") + 1).log()).round(2).alias("score_log"),
            # 방법 2: sqrt(C_recent) 사용
            (pl.col("ratio") * pl.col("C_recent").sqrt()).round(2).alias("score_sqrt"),
            
            # 상대적 증가율 탐지(랭킹 1)
            # 가중 점수: score_ratio = log(ratio) * log(C_recent + 1)
            (pl.col("ratio").log() * (pl.col("C_recent") + 1).log()).round(2).alias("score_ratio"),
        ])
        .sort(["keyword", "window"])
    )
    
    step6_time = time.time() - step6_start
    total_time = time.time() - start_time
    
    print(f"[시간 측정] 전체 실행 시간: {total_time:.3f}초 ({total_time/60:.2f}분)")
    print(f"  - 모든 keyword 목록 수집: {step0_time:.3f}초 ({step0_time/total_time*100:.1f}%)")
    print(f"  - 1개월 키워드 집계: {step1_time:.3f}초 ({step1_time/total_time*100:.1f}%)")
    print(f"  - 1개월 전체 보고서 집계: {step2_time:.3f}초 ({step2_time/total_time*100:.1f}%)")
    print(f"  - 3개월 키워드 집계: {step3_time:.3f}초 ({step3_time/total_time*100:.1f}%)")
    print(f"  - 3개월 전체 보고서 집계: {step4_time:.3f}초 ({step4_time/total_time*100:.1f}%)")
    print(f"  - 데이터 준비 및 통합: {step5_time:.3f}초 ({step5_time/total_time*100:.1f}%)")
    print(f"  - 최종 통합 및 계산: {step6_time:.3f}초 ({step6_time/total_time*100:.1f}%)")
    
    # LazyFrame으로 변환하여 반환
    result = baseline_final.lazy()
    
    return result

In [5]:
# 통합 베이스라인 집계 함수 실행
lf_baseline_final = create_complete_baseline_table(
    lf,
    as_of_month='2025-11',  # None이면 자동으로 최신 월 사용
    spike_threshold_ratio=2.0
)

print("\n통합 베이스라인 집계 결과:")
baseline_final = lf_baseline_final.collect().to_pandas()
display(baseline_final)

# score_log  (소프트 가중 1 -> 건수가 많을수록 우선순위)
# score_sqrt (소프트 가중 2 -> 건수가 많을수록 우선순위)
# score_ratio (가중 점수 -> 배수증가, 최근 절대 건수 증가)


기준 월: 2025-11
최근 1개월: ['2025-11']
최근 3개월: ['2025-11', '2025-10', '2025-09']
기준 1개월: ['2025-10']
기준 3개월: ['2025-10', '2025-09', '2025-08']
[시간 측정] 모든 keyword 목록 수집: 0.102초
총 keyword 개수: 13
[시간 측정] 3개월 전체 보고서 집계: 0.036초
[시간 측정] 전체 실행 시간: 0.233초 (0.00분)
  - 모든 keyword 목록 수집: 0.102초 (43.6%)
  - 1개월 키워드 집계: 0.025초 (10.5%)
  - 1개월 전체 보고서 집계: 0.022초 (9.5%)
  - 3개월 키워드 집계: 0.047초 (20.0%)
  - 3개월 전체 보고서 집계: 0.036초 (15.3%)
  - 데이터 준비 및 통합: 0.001초 (0.4%)
  - 최종 통합 및 계산: 0.001초 (0.3%)

통합 베이스라인 집계 결과:


/var/folders/qk/10b0q2j971j6d2py2x39ys200000gn/T/ipykernel_50538/3421434177.py:237: DeprecationWarning: use of `how='outer'` should be replaced with `how='full'`.
(Deprecated in version 0.20.29)
  recent_3m_keyword


,as_of_month,window,keyword,C_recent,C_base,N_recent,N_base,ratio,is_spike,score_log,score_sqrt,score_ratio
0,2025-11,1,Alarm/Alert,92,65,13241,11522,1.41,False,6.39,13.52,1.56
1,2025-11,3,Alarm/Alert,223,194,35641,36259,1.15,False,6.22,17.17,0.76
2,2025-11,1,Communication/Connectivity,354,328,13241,11522,1.08,False,6.34,20.32,0.45
3,2025-11,3,Communication/Connectivity,949,944,35641,36259,1.01,False,6.93,31.11,0.07
4,2025-11,1,Electrical/Power,1128,1012,13241,11522,1.11,False,7.80,37.28,0.73
5,2025-11,3,Electrical/Power,2804,2530,35641,36259,1.11,False,8.81,58.78,0.83
6,2025-11,1,Environmental/Compatibility,86,101,13241,11522,0.85,False,3.80,7.88,-0.73
7,2025-11,3,Environmental/Compatibility,288,325,35641,36259,0.89,False,5.04,15.10,-0.66
8,2025-11,1,Functional Failure,990,830,13241,11522,1.19,False,8.21,37.44,1.20
9,2025-11,3,Functional Failure,2637,2696,35641,36259,0.98,False,7.72,50.32,-0.16


## z-score와 Poisson 기반으로 이상탐지하는 함수 만들기(두 개 따로 만들어서 합치기!)

In [1]:
import polars as pl
from typing import Literal

def detect_spike_by_z_log_score(
    lf: pl.LazyFrame,
    window: Literal[1, 3] = 1,
    z_threshold: float = 2.0,
    min_count_threshold: int = 5,
    eps: float = 0.1
) -> pl.LazyFrame:
    """
    Z-score 기반 키워드 스파이크 탐지 (log 변환 + 최소 보고서 수 필터)
    
    Parameters
    ----------
    lf : pl.LazyFrame
        키워드별 윈도우 집계 결과
        필수 컬럼: as_of_month, window, keyword, C_recent, N_recent, C_base, N_base
    window : Literal[1, 3], default=1
        분석할 윈도우 크기 (1개월 또는 3개월)
    z_threshold : float, default=2.0
        스파이크 판단 기준 z-score 임계값
    min_count_threshold : int, default=5
        최소 보고서 수 (이보다 적으면 노이즈로 간주)
    eps : float, default=0.1
        분모가 0이 되는 것을 방지하기 위한 작은 값
    
    Returns
    -------
    pl.LazyFrame
        스파이크 탐지 결과
        - log_recent: log(C_recent + 1)
        - log_base_mean: log(C_base + 1)
        - log_base_std: 표준편차 (단일 기준구간이므로 0)
        - z_log: 계산된 z-score
        - is_spike: (z_log >= z_threshold) & (C_recent >= min_count_threshold)
    
    Notes
    -----
    - σ=0 문제 해결: 단일 기준구간 비교이므로 std=0, eps를 더해 처리
    - log 변환: 보고서 수의 급격한 증가를 안정적으로 포착
    - 노이즈 필터링: min_count_threshold로 적은 보고서 수는 제외
    """
    
    result = (
        lf
        # 지정된 윈도우만 필터링
        .filter(pl.col("window") == window)
        
        # log 변환된 값 계산
        .with_columns([
            (pl.col("C_recent") + 1).log().alias("log_recent"),
            (pl.col("C_base") + 1).log().alias("log_base_mean"),
            pl.lit(0.0).alias("log_base_std")
        ])
        
        # Z-score 계산
        .with_columns([
            (
                (pl.col("log_recent") - pl.col("log_base_mean")) / 
                (pl.col("log_base_std") + eps)
            ).alias("z_log")
        ])
        
        # 스파이크 여부 판단 (복합 기준)
        .with_columns([
            (
                (pl.col("z_log") >= z_threshold) &
                (pl.col("C_recent") >= min_count_threshold)
            ).alias("is_spike")
        ])
    )
    
    return result

In [ ]:
import polars as pl
from datetime import datetime
from dateutil.relativedelta import relativedelta
from typing import Optional, Tuple, List, Literal, Union


class BaselineAggregator:
    """
    키워드별 베이스라인 집계 및 스파이크 탐지를 위한 클래스
    
    최근 구간과 기준 구간을 비교하여 키워드별 보고서 수의 변화를 분석합니다.
    """
    
    def __init__(self, lf: pl.LazyFrame):
        """
        Parameters
        ----------
        lf : pl.LazyFrame
            원본 보고서 데이터 (필수 컬럼: mdr_report_key, date_received, defect_type)
        """
        self.lf = lf
        self._lf_keyword_monthly: Optional[pl.LazyFrame] = None
        self._lf_monthly_total: Optional[pl.LazyFrame] = None
        
    def _prepare_monthly_data(self) -> None:
        """월별 집계 데이터를 준비합니다."""
        lf_with_month = self.lf.with_columns(
            pl.col("date_received").cast(pl.Date).dt.strftime("%Y-%m").alias("as_of_month")
        )
        
        # 키워드 테이블 생성
        lf_keyword_table = (
            lf_with_month
            .filter(pl.col("defect_type").is_not_null())
            .select(
                pl.col("mdr_report_key"),
                pl.col("defect_type").alias("keyword"),
                pl.col("as_of_month").alias("month")
            )
            .unique(subset=["mdr_report_key", "keyword", "month"])
        )
        
        # 월별 키워드별 집계
        self._lf_keyword_monthly = (
            lf_keyword_table
            .group_by(["keyword", "month"])
            .agg(pl.col("mdr_report_key").n_unique().alias("n_reports_keyword"))
            .sort(["keyword", "month"])
        )
        
        # 월별 전체 보고서 수 집계
        self._lf_monthly_total = (
            self.lf
            .filter(pl.col("date_received").is_not_null())
            .with_columns(
                pl.col("date_received").cast(pl.Date).dt.strftime("%Y-%m").alias("month")
            )
            .group_by("month")
            .agg(pl.len().alias("n_total_reports"))
            .sort("month")
        )
    
    def _get_latest_month(self) -> str:
        """가장 최신 월을 반환합니다."""
        available_months = (
            self._lf_monthly_total
            .select("month")
            .sort("month", descending=True)
            .collect()
            .to_series()
            .to_list()
        )
        if not available_months:
            raise ValueError("사용 가능한 월 데이터가 없습니다.")
        return available_months[0]
    
    def _calculate_time_windows(
        self, 
        as_of_month: str
    ) -> Tuple[List[str], List[str], List[str], List[str]]:
        """
        최근 구간과 기준 구간의 월 리스트를 계산합니다.
        
        Returns
        -------
        tuple
            (recent_1month, base_1month, recent_3month, base_3month)
        """
        as_of_date = datetime.strptime(as_of_month, "%Y-%m")
        
        recent_1month = [as_of_month]
        recent_3month = [
            (as_of_date - relativedelta(months=i)).strftime("%Y-%m")
            for i in range(3)
        ]
        base_1month = [(as_of_date - relativedelta(months=1)).strftime("%Y-%m")]
        base_3month = [
            (as_of_date - relativedelta(months=i)).strftime("%Y-%m")
            for i in range(1, 4)
        ]
        
        return recent_1month, base_1month, recent_3month, base_3month
    
    def _aggregate_keyword_window(
        self,
        recent_months: List[str],
        base_months: List[str],
        all_keywords: pl.DataFrame,
        window: int
    ) -> pl.DataFrame:
        """특정 윈도우에 대해 키워드별 보고서 수를 집계합니다."""
        recent_df = (
            self._lf_keyword_monthly
            .filter(pl.col("month").is_in(recent_months))
            .group_by("keyword")
            .agg(pl.col("n_reports_keyword").sum().alias(f"C_recent_{window}m"))
            .collect()
        )
        
        base_df = (
            self._lf_keyword_monthly
            .filter(pl.col("month").is_in(base_months))
            .group_by("keyword")
            .agg(pl.col("n_reports_keyword").sum().alias(f"C_base_{window}m"))
            .collect()
        )
        
        return (
            all_keywords
            .join(recent_df, on="keyword", how="left")
            .join(base_df, on="keyword", how="left")
            .with_columns(
                pl.col(f"C_recent_{window}m").fill_null(0),
                pl.col(f"C_base_{window}m").fill_null(0)
            )
        )
    
    def _aggregate_total_window(
        self,
        recent_months: List[str],
        base_months: List[str]
    ) -> Tuple[int, int]:
        """특정 윈도우에 대해 전체 보고서 수를 집계합니다."""
        N_recent = (
            self._lf_monthly_total
            .filter(pl.col("month").is_in(recent_months))
            .select(pl.sum('n_total_reports'))
            .collect()
            .item()
        ) or 0
        
        N_base = (
            self._lf_monthly_total
            .filter(pl.col("month").is_in(base_months))
            .select(pl.sum('n_total_reports'))
            .collect()
            .item()
        ) or 0
        
        return N_recent, N_base
    
    def _calculate_metrics(
        self,
        df: pl.DataFrame,
        z_threshold: float,
        min_c_recent: int,
        eps: float,
        alpha: float,
        correction_method: Optional[str]
    ) -> pl.DataFrame:
        """
        ratio, score, z_log, Poisson 등의 지표를 계산합니다.
        
        Parameters
        ----------
        alpha : float
            유의수준 (예: 0.05, 0.01, 0.001)
        correction_method : str or None
            다중검정 보정법
            - None: 보정 없음 (raw p-value 사용)
            - "bonferroni": Bonferroni 보정
            - "sidak": Šidák 보정
            - "fdr_bh": Benjamini-Hochberg FDR
        """
        from scipy import stats
        import numpy as np
        
        result = (
            df
            # ratio 계산
            .with_columns(
                ((pl.col("C_recent") + 1) / (pl.col("C_base") + 1))
                .round(2)
                .alias("ratio")
            )
            # score 계산
            .with_columns(
                # log 가중
                (pl.col("ratio") * (pl.col("C_recent") + 1).log())
                .round(2)
                .alias("score_log"),
                # sqrt 가중
                (pl.col("ratio") * pl.col("C_recent").sqrt())
                .round(2)
                .alias("score_sqrt"),
                # 상대적 증가율
                (pl.col("ratio").log() * (pl.col("C_recent") + 1).log())
                .round(2)
                .alias("score_ratio")
            )
            # z_log 계산
            .with_columns(
                (pl.col("C_recent") + 1).log().alias("log_recent"),
                (pl.col("C_base") + 1).log().alias("log_base_mean")
            )
            .with_columns(
                ((pl.col("log_recent") - pl.col("log_base_mean")) / eps)
                .round(4)
                .alias("z_log")
            )
            # is_spike_z: z_log 기반 스파이크 판단
            .with_columns(
                (
                    (pl.col("z_log") >= z_threshold) &
                    (pl.col("C_recent") >= min_c_recent)
                ).alias("is_spike_z")
            )
            # is_spike: score_ratio 기반 스파이크 판단
            .with_columns(
                pl.col("score_ratio").mean().over("window").alias("_score_ratio_mean"),
                pl.col("score_ratio").std().over("window").alias("_score_ratio_std")
            )
            .with_columns(
                (
                    (pl.col("score_ratio") >= 
                     (pl.col("_score_ratio_mean") + z_threshold * pl.col("_score_ratio_std"))) &
                    (pl.col("C_recent") >= min_c_recent)
                ).alias("is_spike")
            )
            .drop(["_score_ratio_mean", "_score_ratio_std"])
            # Poisson lambda 계산: λ = (C_base / N_base) * N_recent
            .with_columns(
                pl.when(pl.col("N_base") > 0)
                .then((pl.col("C_base") / pl.col("N_base")) * pl.col("N_recent"))
                .otherwise(pl.col("C_base").cast(pl.Float64))
                .alias("lambda_pois")
            )
            .with_columns(
                pl.when(pl.col("lambda_pois") < 0.001)
                .then(0.001)
                .otherwise(pl.col("lambda_pois"))
                .alias("lambda_pois")
            )
        )
        
        # Poisson p-value 계산
        c_recent = result["C_recent"].to_numpy()
        lambda_pois = result["lambda_pois"].to_numpy()
        
        # P(X >= k | λ) = 1 - P(X <= k-1 | λ)
        p_pois = np.where(
            c_recent > 0,
            1 - stats.poisson.cdf(c_recent - 1, lambda_pois),
            1.0
        )
        p_pois = np.maximum(p_pois, 1e-300)
        score_pois = -np.log10(p_pois)
        
        # 다중검정 보정
        n = len(p_pois)
        if correction_method is None:
            # 보정 없음
            p_adjusted = p_pois
            alpha_adjusted = alpha
        elif correction_method == "bonferroni":
            # Bonferroni: α / n
            p_adjusted = np.minimum(p_pois * n, 1.0)
            alpha_adjusted = alpha
        elif correction_method == "sidak":
            # Šidák: 1 - (1-α)^(1/n)
            p_adjusted = 1 - (1 - p_pois) ** n
        elif correction_method == "fdr_bh":
            # Benjamini-Hochberg FDR - 직접 구현
            # 1. p-value 정렬
            sorted_indices = np.argsort(p_pois)
            sorted_p = p_pois[sorted_indices]
            
            # 2. BH adjusted p-value 계산
            # adjusted_p[i] = min(p[i] * n / rank[i], 1)
            # 그리고 뒤에서부터 cumulative minimum 적용
            ranks = np.arange(1, n + 1)
            adjusted_sorted = np.minimum(sorted_p * n / ranks, 1.0)
            
            # 뒤에서부터 cumulative minimum (monotonicity 보장)
            for i in range(n - 2, -1, -1):
                adjusted_sorted[i] = min(adjusted_sorted[i], adjusted_sorted[i + 1])
            
            # 3. 원래 순서로 복원
            p_adjusted = np.empty(n)
            p_adjusted[sorted_indices] = adjusted_sorted
        else:
            raise ValueError(f"Unknown correction method: {correction_method}")
        
        # 결과 추가
        result = result.with_columns(
            pl.Series("p_pois", p_pois).round(6),
            pl.Series("p_adjusted", p_adjusted).round(6),
            pl.Series("score_pois", score_pois).round(4)
        )
        
        # is_spike_p 계산: adjusted p-value와 alpha 비교
        result = result.with_columns(
            (
                (pl.col("p_adjusted") <= alpha) &
                (pl.col("C_recent") >= min_c_recent)
            ).alias("is_spike_p")
        )
        
        # 컬럼 순서 재배치: is_spike 시리즈를 마지막으로
        spike_cols = ["is_spike", "is_spike_z", "is_spike_p"]
        other_cols = [c for c in result.columns if c not in spike_cols]
        result = result.select(other_cols + spike_cols)
        
        return result.sort(["keyword", "window"])
    
    def _create_window_baseline(
        self,
        keyword_df: pl.DataFrame,
        as_of_month: str,
        window: int,
        N_recent: int,
        N_base: int,
        z_threshold: float,
        min_c_recent: int,
        eps: float,
        alpha: float,
        correction_method: Optional[str]
    ) -> pl.DataFrame:
        """
        윈도우별 베이스라인 데이터프레임을 생성합니다.
        
        모든 지표(ratio, score, z_log, Poisson, is_spike 등)를 포함합니다.
        """
        c_recent_col = f"C_recent_{window}m"
        c_base_col = f"C_base_{window}m"
        
        base_df = (
            keyword_df
            .with_columns(
                pl.lit(as_of_month).alias("as_of_month"),
                pl.lit(window).alias("window"),
                pl.col(c_recent_col).fill_null(0).cast(pl.Int64).alias("C_recent"),
                pl.col(c_base_col).fill_null(0).cast(pl.Int64).alias("C_base"),
                pl.lit(N_recent).cast(pl.Int64).alias("N_recent"),
                pl.lit(N_base).cast(pl.Int64).alias("N_base")
            )
            .select([
                "as_of_month", "window", "keyword",
                "C_recent", "C_base", "N_recent", "N_base"
            ])
        )
        
        # 지표 계산 적용
        return self._calculate_metrics(
            base_df, z_threshold, min_c_recent, eps, alpha, correction_method
        )
    
    def create_baseline_table(
        self,
        as_of_month: Optional[str] = None,
        z_threshold: float = 2.0,
        min_c_recent: int = 20,
        eps: float = 0.1,
        alpha: float = 0.05,
        correction_method: Optional[Literal["bonferroni", "sidak", "fdr_bh"]] = "fdr_bh",
        verbose: bool = True
    ) -> pl.LazyFrame:
        """
        베이스라인 집계 테이블을 Long Format으로 생성합니다.
        
        Parameters
        ----------
        as_of_month : str, optional
            기준 월 (예: "2025-11"). None이면 자동으로 최신 월 사용
        z_threshold : float, default=2.0
            스파이크 판단 기준 z-score 임계값
        min_c_recent : int, default=20
            스파이크 판단 최소 C_recent 값
        eps : float, default=0.1
            z_log 계산 시 분모에 더할 작은 값
        alpha : float, default=0.05
            Poisson 스파이크 판단 유의수준
            - 0.05: 일반적 유의수준 (*)
            - 0.01: 매우 유의 (**)
            - 0.001: 고도로 유의 (***)
        correction_method : str or None, default="fdr_bh"
            다중검정 보정법
            - None: 보정 없음 (raw p-value 사용)
            - "bonferroni": Bonferroni 보정 (가장 보수적)
            - "sidak": Šidák 보정
            - "fdr_bh": Benjamini-Hochberg FDR (권장, 거짓발견율 제어)
        verbose : bool, default=True
            진행 상황 출력 여부
            
        Returns
        -------
        pl.LazyFrame
            Long Format 베이스라인 집계 테이블
            컬럼: as_of_month, window(int), keyword, C_recent, C_base, N_recent, N_base,
                  ratio, score_log, score_sqrt, score_ratio, 
                  log_recent, log_base_mean, z_log, is_spike_z, is_spike,
                  lambda_pois, p_pois, p_adjusted, score_pois, is_spike_p
        """
        if verbose:
            print("월별 집계 데이터 준비 중...")
        self._prepare_monthly_data()
        
        if as_of_month is None:
            as_of_month = self._get_latest_month()
            if verbose:
                print(f"자동으로 최신 월을 사용합니다: {as_of_month}")
        
        recent_1m, base_1m, recent_3m, base_3m = self._calculate_time_windows(as_of_month)
        
        if verbose:
            print(f"\n기준 월: {as_of_month}")
            print(f"최근 1개월: {recent_1m}, 기준 1개월: {base_1m}")
            print(f"최근 3개월: {recent_3m}, 기준 3개월: {base_3m}")
            print(f"\nPoisson 설정: alpha={alpha}, correction={correction_method}")
        
        all_keywords = self._lf_keyword_monthly.select("keyword").unique().collect()
        
        if verbose:
            print(f"총 keyword 개수: {len(all_keywords)}")
        
        # 윈도우별 집계
        keyword_1m = self._aggregate_keyword_window(recent_1m, base_1m, all_keywords, 1)
        N_recent_1m, N_base_1m = self._aggregate_total_window(recent_1m, base_1m)
        
        keyword_3m = self._aggregate_keyword_window(recent_3m, base_3m, all_keywords, 3)
        N_recent_3m, N_base_3m = self._aggregate_total_window(recent_3m, base_3m)
        
        # 베이스라인 생성 (지표 포함)
        baseline_1m = self._create_window_baseline(
            keyword_1m, as_of_month, 1, N_recent_1m, N_base_1m,
            z_threshold, min_c_recent, eps, alpha, correction_method
        )
        baseline_3m = self._create_window_baseline(
            keyword_3m, as_of_month, 3, N_recent_3m, N_base_3m,
            z_threshold, min_c_recent, eps, alpha, correction_method
        )
        
        baseline_final = pl.concat([baseline_1m, baseline_3m]).sort(["keyword", "window"])
        
        if verbose:
            print("\n베이스라인 테이블 생성 완료")
        
        return baseline_final.lazy()
    
    def detect_spike_by_z_log_score(
        self,
        baseline_lf: pl.LazyFrame,
        window: Union[int, Literal[1, 3]] = 1
    ) -> pl.LazyFrame:
        """
        Z-score 기반 키워드 스파이크 탐지 결과를 반환합니다.
        
        Parameters
        ----------
        baseline_lf : pl.LazyFrame
            create_baseline_table()로 생성된 베이스라인 테이블
        window : int, default=1
            분석할 윈도우 크기 (1 또는 3)
        
        Returns
        -------
        pl.LazyFrame
            해당 윈도우의 z_log 기반 스파이크 탐지 결과
        """
        return (
            baseline_lf
            .filter(pl.col("window") == window)
            .filter(pl.col("is_spike_z") == True)
            .sort("z_log", descending=True)
        )
    
    def detect_spike_by_poisson(
        self,
        baseline_lf: pl.LazyFrame,
        window: Union[int, Literal[1, 3]] = 1
    ) -> pl.LazyFrame:
        """
        Poisson 분포 기반 키워드 스파이크 탐지 결과를 반환합니다.
        
        Parameters
        ----------
        baseline_lf : pl.LazyFrame
            create_baseline_table()로 생성된 베이스라인 테이블
        window : int, default=1
            분석할 윈도우 크기 (1 또는 3)
        
        Returns
        -------
        pl.LazyFrame
            해당 윈도우의 Poisson 기반 스파이크 탐지 결과
            - lambda_pois: Poisson 기대치 λ = (C_base / N_base) * N_recent
            - p_pois: P(X >= k | λ), tail probability
            - score_pois: -log10(p_pois), 이상도 점수
            - is_spike_p: 스파이크 여부
        """
        return (
            baseline_lf
            .filter(pl.col("window") == window)
            .filter(pl.col("is_spike_p") == True)
            .sort("score_pois", descending=True)
        )
    
    def detect_spikes(
        self,
        baseline_lf: pl.LazyFrame,
        window: Union[int, Literal[1, 3]] = 1,
        spike_type: Literal["ratio", "z", "poisson"] = "ratio"
    ) -> pl.LazyFrame:
        """
        스파이크 키워드만 필터링하여 반환합니다.
        
        Parameters
        ----------
        baseline_lf : pl.LazyFrame
            create_baseline_table()로 생성된 베이스라인 테이블
        window : int, default=1
            분석할 윈도우 크기 (1 또는 3)
        spike_type : Literal["ratio", "z", "poisson"], default="ratio"
            "ratio": score_ratio 기반 is_spike
            "z": z_log 기반 is_spike_z
            "poisson": Poisson 기반 is_spike_p
        
        Returns
        -------
        pl.LazyFrame
            스파이크로 판단된 키워드만 포함
        """
        spike_map = {
            "ratio": ("is_spike", "score_ratio"),
            "z": ("is_spike_z", "z_log"),
            "poisson": ("is_spike_p", "score_pois")
        }
        spike_col, sort_col = spike_map[spike_type]
        
        return (
            baseline_lf
            .filter(
                (pl.col("window") == window) &
                (pl.col(spike_col) == True)
            )
            .sort(sort_col, descending=True)
        )
    
    def detect_spike_ensemble(
        self,
        baseline_lf: pl.LazyFrame,
        window: Union[int, Literal[1, 3]] = 1,
        min_methods: int = 2,
        min_c_recent: Optional[int] = None
    ) -> pl.LazyFrame:
        """
        앙상블 기반 스파이크 탐지 (다중 방법 통합)
        
        절대빈도 가드를 통과한 후, 여러 탐지 방법 중 
        min_methods개 이상 만족 시 최종 스파이크로 판정합니다.
        
        Parameters
        ----------
        baseline_lf : pl.LazyFrame
            create_baseline_table()로 생성된 베이스라인 테이블
        window : int, default=1
            분석할 윈도우 크기 (1 또는 3)
        min_methods : int, default=2
            스파이크 판정에 필요한 최소 방법 수 (1~3)
            - 1: 하나라도 만족하면 스파이크 (OR 조건, 민감)
            - 2: 2개 이상 만족 시 스파이크 (권장)
            - 3: 모두 만족해야 스파이크 (AND 조건, 보수적)
        min_c_recent : int, optional
            절대빈도 가드 (C_recent 최소값)
            None이면 baseline_table 생성 시 사용한 값 적용
        
        Returns
        -------
        pl.LazyFrame
            앙상블 스파이크 탐지 결과
            추가 컬럼:
            - n_methods: 만족한 방법 수 (0~3)
            - is_spike_ensemble: 최종 앙상블 스파이크 여부
        
        Notes
        -----
        3가지 방법:
        - (A) ratio 기반: is_spike (score_ratio 기준)
        - (B) z-score 기반: is_spike_z (z_log 기준)
        - (C) Poisson 기반: is_spike_p (p_pois 기준, 다중검정 보정 적용)
        """
        result = (
            baseline_lf
            .filter(pl.col("window") == window)
            # 만족한 방법 수 계산
            .with_columns(
                (
                    pl.col("is_spike").cast(pl.Int8) +
                    pl.col("is_spike_z").cast(pl.Int8) +
                    pl.col("is_spike_p").cast(pl.Int8)
                ).alias("n_methods")
            )
        )
        
        # 절대빈도 가드 적용
        if min_c_recent is not None:
            result = result.filter(pl.col("C_recent") >= min_c_recent)
        
        # 앙상블 스파이크 판정
        result = (
            result
            .with_columns(
                (pl.col("n_methods") >= min_methods).alias("is_spike_ensemble")
            )
            # 컬럼 순서 재배치: is_spike 시리즈를 마지막으로
            .select(
                pl.exclude(["is_spike", "is_spike_z", "is_spike_p", 
                           "n_methods", "is_spike_ensemble"]),
                "n_methods",
                "is_spike",
                "is_spike_z", 
                "is_spike_p",
                "is_spike_ensemble"
            )
            .sort("n_methods", descending=True)
        )
        
        return result
    
    def get_ensemble_spikes(
        self,
        baseline_lf: pl.LazyFrame,
        window: Union[int, Literal[1, 3]] = 1,
        min_methods: int = 2,
        min_c_recent: Optional[int] = None
    ) -> pl.LazyFrame:
        """
        앙상블 기준으로 스파이크인 키워드만 반환합니다.
        
        Parameters
        ----------
        baseline_lf : pl.LazyFrame
            create_baseline_table()로 생성된 베이스라인 테이블
        window : int, default=1
            분석할 윈도우 크기 (1 또는 3)
        min_methods : int, default=2
            스파이크 판정에 필요한 최소 방법 수
        min_c_recent : int, optional
            절대빈도 가드
        
        Returns
        -------
        pl.LazyFrame
            앙상블 스파이크로 판정된 키워드만 포함
        """
        return (
            self.detect_spike_ensemble(
                baseline_lf, 
                window=window, 
                min_methods=min_methods,
                min_c_recent=min_c_recent
            )
            .filter(pl.col("is_spike_ensemble") == True)
            .sort(["n_methods", "score_pois"], descending=True)
        )

In [21]:
aggregator = BaselineAggregator(lf)

baseline_lf = aggregator.create_baseline_table(
    as_of_month="2025-10",          # None이면 최신 월 자동 선택
    z_threshold=2.0,                # 스파이크 판단 z-score 임계값
    min_c_recent=20,                # 최소 보고서 수
    eps=0.1,                        # z_log 분모 보정값
    alpha=0.05,                     # 유의수준 선택
    correction_method="bonferroni", # 다중검정 보정법 선택
    verbose=True
)

baseline_lf.collect().to_pandas()

월별 집계 데이터 준비 중...

기준 월: 2025-10
최근 1개월: ['2025-10'], 기준 1개월: ['2025-09']
최근 3개월: ['2025-10', '2025-09', '2025-08'], 기준 3개월: ['2025-09', '2025-08', '2025-07']

Poisson 설정: alpha=0.05, correction=bonferroni
총 keyword 개수: 13

베이스라인 테이블 생성 완료


,as_of_month,window,keyword,C_recent,C_base,N_recent,N_base,ratio,score_log,score_sqrt,...,log_recent,log_base_mean,z_log,lambda_pois,p_pois,p_adjusted,score_pois,is_spike,is_spike_z,is_spike_p
0,2025-10,1,Alarm/Alert,65,66,11522,10878,0.99,4.15,7.98,...,4.189655,4.204693,-0.1504,69.907336,0.737322,1.000000,0.1323,False,False,False
1,2025-10,3,Alarm/Alert,194,176,36259,40999,1.10,5.80,15.32,...,5.273000,5.176150,0.9685,155.652187,0.001670,0.021711,2.7773,True,False,True
2,2025-10,1,Communication/Connectivity,328,267,11522,10878,1.23,7.13,22.28,...,5.796058,5.590987,2.0507,282.806950,0.004640,0.060323,2.3335,False,True,False
3,2025-10,3,Communication/Connectivity,944,976,36259,40999,0.97,6.65,29.80,...,6.851185,6.884487,-0.3330,863.162126,0.003475,0.045178,2.4590,False,False,True
4,2025-10,1,Electrical/Power,1012,664,11522,10878,1.52,10.52,48.35,...,6.920672,6.499787,4.2088,703.310167,0.000000,0.000000,300.0000,True,True,True
5,2025-10,3,Electrical/Power,2530,2515,36259,40999,1.01,7.91,50.80,...,7.836370,7.830426,0.0594,2224.234372,0.000000,0.000000,9.9285,False,False,True
6,2025-10,1,Environmental/Compatibility,101,101,11522,10878,1.00,4.62,10.05,...,4.624973,4.624973,0.0000,106.979408,0.731263,1.000000,0.1359,False,False,False
7,2025-10,3,Environmental/Compatibility,325,387,36259,40999,0.84,4.86,15.14,...,5.786897,5.961005,-1.7411,342.257933,0.831289,1.000000,0.0802,False,False,False
8,2025-10,1,Functional Failure,830,817,11522,10878,1.02,6.86,29.39,...,6.722630,6.706862,0.1577,865.368082,0.889176,1.000000,0.0510,False,False,False
9,2025-10,3,Functional Failure,2696,3125,36259,40999,0.86,6.79,44.65,...,7.899895,8.047510,-1.4761,2763.710700,0.903149,1.000000,0.0442,False,False,False


In [ ]:
# 앙상블 판정 결과 전체 조회
ensemble_result = aggregator.detect_spike_ensemble(
    baseline_lf,
    window=1,
    min_methods=2
)

ensemble_result.collect().to_pandas()

,as_of_month,window,keyword,C_recent,C_base,N_recent,N_base,ratio,score_log,score_sqrt,...,z_log,lambda_pois,p_pois,p_adjusted,score_pois,n_methods,is_spike,is_spike_z,is_spike_p,is_spike_ensemble
0,2025-10,1,Electrical/Power,1012,664,11522,10878,1.52,10.52,48.35,...,4.2088,703.310167,0.000000,0.000000,300.0000,3,True,True,True,True
1,2025-10,1,Mechanical/Structural,2215,1787,11522,10878,1.24,9.55,58.36,...,2.1461,1892.794080,0.000000,0.000000,12.5287,2,False,True,True,True
2,2025-10,1,Communication/Connectivity,328,267,11522,10878,1.23,7.13,22.28,...,2.0507,282.806950,0.004640,0.060323,2.3335,1,False,True,False,False
3,2025-10,1,User/Human Factor,113,91,11522,10878,1.24,5.87,13.18,...,2.1441,96.387387,0.053164,0.691126,1.2744,1,False,True,False,False
4,2025-10,1,Alarm/Alert,65,66,11522,10878,0.99,4.15,7.98,...,-0.1504,69.907336,0.737322,1.000000,0.1323,0,False,False,False,False
5,2025-10,1,Environmental/Compatibility,101,101,11522,10878,1.00,4.62,10.05,...,0.0000,106.979408,0.731263,1.000000,0.1359,0,False,False,False,False
6,2025-10,1,Functional Failure,830,817,11522,10878,1.02,6.86,29.39,...,0.1577,865.368082,0.889176,1.000000,0.0510,0,False,False,False,False
7,2025-10,1,Labeling/Packaging,14,25,11522,10878,0.58,1.57,2.17,...,-5.5005,26.480051,0.997032,1.000000,0.0013,0,False,False,False,False
8,2025-10,1,Other,2745,2656,11522,10878,1.03,8.16,53.96,...,0.3295,2813.240669,0.902889,1.000000,0.0444,0,False,False,False,False
9,2025-10,1,Sensor/Accuracy,2342,2626,11522,10878,0.89,6.91,43.07,...,-1.1441,2781.464607,1.000000,1.000000,-0.0000,0,False,False,False,False


In [27]:
# 앙상블 스파이크만 조회
spikes = aggregator.get_ensemble_spikes(
    baseline_lf,
    window=3,
    min_methods=2,
    min_c_recent=20  # 추가 필터
)

spikes.collect().to_pandas()

,as_of_month,window,keyword,C_recent,C_base,N_recent,N_base,ratio,score_log,score_sqrt,...,z_log,lambda_pois,p_pois,p_adjusted,score_pois,n_methods,is_spike,is_spike_z,is_spike_p,is_spike_ensemble
0,2025-10,3,Alarm/Alert,194,176,36259,40999,1.1,5.8,15.32,...,0.9685,155.652187,0.00167,0.021711,2.7773,2,True,False,True,True


In [3]:
import sys
from pathlib import Path

# 상대 경로 사용
PROJECT_ROOT = Path.cwd().parent
DATA_DIR = PROJECT_ROOT / 'data'

# 맨 앞에 추가
sys.path.insert(0, str(PROJECT_ROOT))

# Python 내장 모듈 캐시만 임시 제거
if 'src' in sys.modules:
    del sys.modules['src']

In [4]:
from src.utils.baseline_aggregator import BaselineAggregator

aggregator = BaselineAggregator(lf)
baseline_lf = aggregator.create_baseline_table()

월별 집계 데이터 준비 중...
자동으로 최신 월을 사용합니다: 2025-11

기준 월: 2025-11
최근 1개월: ['2025-11'], 기준 1개월: ['2025-10']
최근 3개월: ['2025-11', '2025-10', '2025-09'], 기준 3개월: ['2025-10', '2025-09', '2025-08']

Poisson 설정: alpha=0.05, correction=fdr_bh
총 keyword 개수: 13

베이스라인 테이블 생성 완료


In [5]:
baseline_lf.collect().to_pandas()

,as_of_month,window,keyword,C_recent,C_base,N_recent,N_base,ratio,score_log,score_sqrt,...,log_recent,log_base_mean,z_log,lambda_pois,p_pois,p_adjusted,score_pois,is_spike,is_spike_z,is_spike_p
0,2025-11,1,Alarm/Alert,92,65,13241,11522,1.41,6.39,13.52,...,4.532599,4.189655,3.4294,74.697535,0.028983,0.094195,1.5379,False,True,False
1,2025-11,3,Alarm/Alert,223,194,35641,36259,1.15,6.22,17.17,...,5.411646,5.273000,1.3865,190.693455,0.012063,0.078413,1.9185,False,False,False
2,2025-11,1,Communication/Connectivity,354,328,13241,11522,1.08,6.34,20.32,...,5.872118,5.796058,0.7606,376.935254,0.887099,1.000000,0.0520,False,False,False
3,2025-11,3,Communication/Connectivity,949,944,35641,36259,1.01,6.93,31.11,...,6.856462,6.851185,0.0528,927.910422,0.248603,0.646367,0.6045,False,False,False
4,2025-11,1,Electrical/Power,1128,1012,13241,11522,1.11,7.80,37.28,...,7.029088,6.920672,1.0842,1162.983163,0.851046,1.000000,0.0700,False,False,False
5,2025-11,3,Electrical/Power,2804,2530,35641,36259,1.11,8.81,58.78,...,7.939159,7.836370,1.0279,2486.878568,0.000000,0.000000,9.6166,False,False,True
6,2025-11,1,Environmental/Compatibility,86,101,13241,11522,0.85,3.80,7.88,...,4.465908,4.624973,-1.5906,116.068478,0.998470,1.000000,0.0007,False,False,False
7,2025-11,3,Environmental/Compatibility,288,325,35641,36259,0.89,5.04,15.10,...,5.666427,5.786897,-1.2047,319.460686,0.964837,0.992658,0.0155,False,False,False
8,2025-11,1,Functional Failure,990,830,13241,11522,1.19,8.21,37.44,...,6.898715,6.722630,1.7608,953.830064,0.124407,0.261261,0.9052,False,False,False
9,2025-11,3,Functional Failure,2637,2696,35641,36259,0.98,7.72,50.32,...,7.877776,7.899895,-0.2212,2650.049257,0.602640,0.992658,0.2199,False,False,False


In [7]:
spikes = aggregator.get_ensemble_spikes(
    baseline_lf,
    window=1,
    min_methods=2,
    min_c_recent=20
)

spikes.collect().to_pandas()

,as_of_month,window,keyword,C_recent,C_base,N_recent,N_base,ratio,score_log,score_sqrt,...,z_log,lambda_pois,p_pois,p_adjusted,score_pois,n_methods,is_spike,is_spike_z,is_spike_p,is_spike_ensemble
0,2025-11,1,Other,3380,2745,13241,11522,1.23,9.99,71.51,...,2.0803,3154.534369,0.000037,0.000483,4.4301,2,False,True,True,True
1,2025-11,1,Unknown,1981,1598,13241,11522,1.24,9.41,55.19,...,2.1473,1836.410172,0.000444,0.002888,3.3524,2,False,True,True,True
